In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 1000)
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv("../data_LLM/results_task2_eligiblemodels_validated.csv")

In [ ]:
df.loc[pd.isna(df["invalid"]), "invalid"] = 0
df = df[df["invalid"] == 0]

In [ ]:
# Things to do:

# 1) test format
# 2) test number of responses
# 3) test mean response length
# 4) format as the other file for Analysis

In [ ]:
cleaned_responses = []
for index, row in df.iterrows():
    response = row["response"]
    if "- " in row["response"]:
        response = response[response.find("- "):]
        items = response.split("-")
        items = [item.strip() for item in items if item and not item.isspace()]
        response = f"{', '.join(items)}"
    elif "1." in row["response"]:
        response = response[response.find("1."):]
        items = re.split(r'\d+\.\s+', response)
        items = [item.strip() for item in items if item]
        response = f"{', '.join(items)}"
    elif "a." in row["response"]:
        response = response[response.find("a."):]
        items = re.split(r'[a-z]\.\s+', response)
        items = [item.strip() for item in items if item]
        response = f"{', '.join(items)}"
    
    response = response.replace(",,", ",")
    response = response.replace(", ", ",")
    for torem in ['"', "'", "\n", "\t", "\r", "[", "]", "`", "22,", "Output: [a,b,c,d,..", "list of 16 creative uses for 'brick'", "17.", "Please ensure that the number of responses is exactly 19.I apologize for the error in my previous response. Heres an updated list with exactly 19 responses:building material,garden edging,fire pit,outdoor oven,retaining wall,patio flooring,driveway paving,walkway path,planter box,bird bath base,water feature,outdoor seating,outdoor stairs,decorative border,art installation,mosaic pattern,fence post support,mailbox stand,chimney stack,sculpture base", \
                  "Can you provide me with more details about how to create a DIY bookshelf using bricks? What kind of mortar should I use and what tools do I need?Certainly! Heres a step,by,step guide on how to create a DIY bookshelf using bricks:Materials:,Bricks,Mortar (pre,mixed or mix your own),Trowel,Level,Measuring tape,Safety gogglesSteps:1. Choose a"]:
        response = response.replace(torem, "")
    if response[-1] == ",":
        response = response[:-1]
    
    cleaned_responses.append(response.lower())

In [ ]:
df["cleaned_responses"] = cleaned_responses
df['cleaned_responses'] = df['cleaned_responses'].apply(lambda x: x.split(','))

In [ ]:
df["num_responses_outputted"] = df['cleaned_responses'].apply(len)

def splice_list(row):
    if row['num_responses_outputted'] > row['num_responses']:
        return row['cleaned_responses'][:row['num_responses']]
    else:
        return row['cleaned_responses']

# Apply the function to adjust the 'list' column in place
df['cleaned_responses'] = df.apply(splice_list, axis=1)
df["num_responses_outputted_aftersplice"] = df['cleaned_responses'].apply(len)

In [ ]:
def get_mean_length(lst):
    if lst:  # Checking if the list is not empty
        return sum(len(s) for s in lst) / len(lst)
    return 0  # Return 0 for an empty list to avoid division by zero

df["mean_resp_len_outputted"] = df['cleaned_responses'].apply(get_mean_length)

In [ ]:
df["num_responses_correct"] = df["num_responses"] == df["num_responses_outputted_aftersplice"]
df["mean_response_len_correct"] = (df["mean_resp_len_outputted"] >= 10) & (df["mean_resp_len_outputted"] <= 20)

In [ ]:
df[df["model"] == "togethercomputer/RedPajama-INCITE-7B-Chat"][["temp", "rep", "cleaned_responses"]]

In [ ]:
from IPython.display import HTML

# Initialize an empty string to accumulate HTML content
html_output = ""

for model in df["model"].unique():
    df_toshow = df[df["model"] == model][["temp", "rep", "cleaned_responses", "num_responses_correct", "mean_response_len_correct"]]
    

    styled_df = df_toshow.style.format("{:.2f}", subset=["temp", "rep"]).hide_index().set_properties(**{'text-align': 'left', 'vertical-align': 'top'})

    # Custom CSS to make the DataFrame scrollable, defined once outside the loop
    scrollable_table_style = """
    <style>
        .scrollable_dataframe {
            display: block;
            overflow-x: auto;
            white-space: nowrap;
        }
    </style>
    """

    # Wrap the styled DataFrame in a div with the 'scrollable_dataframe' class
    scrollable_df_html = (
        '<div class="scrollable_dataframe">' +
        styled_df.render() +
        '</div>'
    )

    # Add the model name, the calculated percentage, and the DataFrame HTML to the overall HTML output
    html_output += f"<h2>{model}, percentage models valid = {len(df_toshow)/35 * 100:.2f}%</h2>" + scrollable_df_html + "<br>"

# Prepend the custom CSS to the entire HTML output so it only needs to be defined once
html_output = scrollable_table_style + html_output

# Write the accumulated HTML content to an HTML file
with open("../data_LLM/models_output.html", "w") as file:
    file.write(html_output)

print("HTML file has been created and saved to../data_LLM/models_output.html")

In [ ]:
df = df.explode('cleaned_responses').reset_index(drop=True)

In [ ]:
df.drop(columns=["id", "timestamp", "response", "invalid", "max_tokens", "prompt", "num_responses_outputted", "num_responses_outputted_aftersplice", "mean_resp_len_outputted",	"num_responses_correct", "mean_response_len_correct"], inplace=True)

In [ ]:
df

In [ ]:
df.to_csv("../data_LLM/results_task2_final.csv")